In [2]:
!pip install keras


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
!pip install tensorflow keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.callbacks import TensorBoard
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Embedding, LSTM, Conv1D, Conv2D, MaxPooling1D, Reshape, Flatten, Dropout, CuDNNLSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import operator



import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#from appos import appos

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline


## Loading, extracting and pre-processing the data

Load the data and extract the relevant labels.

In [1]:

import pandas as pd

# Replace 'path_to_your_csv_file.csv' with the actual path to your CSV file on your local system
file_path = '/Users/teja/Downloads/Twitter_Data.csv'

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(data.head())


                                                Text  Score
0  when modi promised “minimum government maximum...    0.0
1  talk all the nonsense and continue all the dra...    0.0
2  what did just say vote for modi  welcome bjp t...    1.0
3  asking his supporters prefix chowkidar their n...    1.0
4  answer who among these the most powerful world...    1.0


In [2]:
# Assume 'data' is your DataFrame with 'Score' column containing 0 and 1 values

# Define the mapping from 0, 1 ratings to negative or positive sentiment
rating_to_sentiment = { 0: 0, 1: 1 }

# Apply the mapping to create the 'Sentiment' column
data['Sentiment'] = data['Score'].map(rating_to_sentiment)

# Count number of negative and positive reviews
neg_num = (data['Sentiment'] == 0).sum()
pos_num = (data['Sentiment'] == 1).sum()

print('# negative reviews before: {}'.format(neg_num))
print('# positive reviews before: {}'.format(pos_num))

# Make the dataset balanced
balanced_sample_num = min(neg_num, pos_num)

# Randomly select <'balanced_sample_num'> numbers of negative and positive reviews
neg_sample = data[data['Sentiment'] == 0].sample(n=balanced_sample_num, replace=False)
pos_sample = data[data['Sentiment'] == 1].sample(n=balanced_sample_num, replace=False)

# Concatenate the samples to create the balanced dataset
data_balanced = pd.concat([neg_sample, pos_sample], ignore_index=True)

# Shuffle the rows so that 0's and 1's are mixed
data_balanced = data_balanced.sample(frac=1).reset_index(drop=True)

print('\n# negative reviews after: {}'.format((data_balanced['Sentiment'] == 0).sum()))
print('# positive reviews after: {}'.format((data_balanced['Sentiment'] == 1).sum()))

# Get one-hot encoding for the labels
Y = pd.get_dummies(data_balanced['Sentiment']).values


# negative reviews before: 90723
# positive reviews before: 72250

# negative reviews after: 72250
# positive reviews after: 72250


#### Perform pre-processing on the data.

In [3]:
pip install --upgrade nltk


Note: you may need to restart the kernel to use updated packages.


In [4]:
import re 
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from transformers import AutoTokenizer
import numpy as np
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd



# Replace 'path_to_your_csv_file.csv' with the actual path to your CSV file on your local system
file_path = '/Users/teja/Downloads/Twitter_Data.csv'

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_path)
# All characters to lower case
data['Text'] = data['Text'].apply(lambda x: x.lower() if isinstance(x, str) else str(x))

# # Convert words with apostrophes to its corresponding words, e.g. "it's" -> "it is"
# data['Text'] = data['Text'].apply(lambda x: x.split())
# data['Text'] = data['Text'].apply(lambda x: " ".join([appos[word] if word in appos else word for word in x]))

# Remove html-tags, punctuation, commas, numbers etc
data['Text'] = data['Text'].apply((lambda x: re.sub('<[^<]+?>', ' ', str(x))))
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', ' ', (x))))
data['Text'] = data['Text'].apply((lambda x: re.sub('^\d+\s|\s\d+\s|\s\d+$', ' ', (x))))

# Convert text into tokens, in this case sentences into words
data['Text'] = data.apply(lambda x: word_tokenize(x['Text']), axis = 1)

# Remove most commonly occuring words which are not relevant in the context of the data
irrelevant_words = stopwords.words('english')
data['Text'] = data['Text'].apply(lambda x: [word for word in x if word not in irrelevant_words])

# Find the base form of the word (lemmatization)
lemma = WordNetLemmatizer()
data['Text'] = data['Text'].apply(lambda x: " ".join([lemma.lemmatize(word) for word in x]))

# Vectorize the text by turning each review into a sequence of integers (each integer being the index of a token in a dictionary)
# Also, pad so that every review has the same length
#num_top_words = 10000
#tokenizer = Tokenizer(num_words = num_top_words, split = ' ')
#tokenizer.fit_on_texts(data['Text'].values)
#X = tokenizer.texts_to_sequences(data['Text'].values)
#X = pad_sequences(X)


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Define the maximum sequence length
max_length = 100  # Adjust as needed

# Tokenize the text data
X = []

for text in data['Text'].values:
    # Tokenize each text and truncate/pad to the specified maximum length
    tokenized_text = tokenizer(text, padding="max_length", truncation=True, max_length=max_length)
    X.append(tokenized_text['input_ids'])

# Convert the tokenized sequences to numpy array
X = np.array(X)


[nltk_data] Downloading package punkt to /Users/teja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/teja/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Print the the <'num_top_words'> number of most frequent words.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Replace 'path_to_your_csv_file.csv' with the actual path to your CSV file on your local system
file_path = '/Users/teja/Downloads/Twitter_Data.csv'

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_path)

# Replace NaN values with an empty string
data['Text'].fillna('', inplace=True)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the text data and transform the text into a matrix of token counts
X = vectorizer.fit_transform(data['Text'])

# Get the vocabulary (i.e., unique words) and their corresponding counts
vocabulary = vectorizer.vocabulary_

# Get the feature names (i.e., the words) from the vocabulary
feature_names = {word: idx for word, idx in sorted(vocabulary.items(), key=lambda x: x[1])}

# Get the top N words by frequency
num_top_words = 10000
top_words_frequency = dict(sorted(feature_names.items(), key=lambda item: X[:, item[1]].sum(), reverse=True)[:num_top_words])

print(top_words_frequency)




{'modi': 62481, 'the': 95482, 'and': 8389, 'for': 37266, 'you': 105957, 'not': 67998, 'india': 47066, 'will': 104111, 'this': 96092, 'that': 95375, 'are': 9779, 'with': 104367, 'has': 42889, 'have': 43151, 'but': 17962, 'all': 7484, 'from': 37936, 'his': 44401, 'was': 103052, 'they': 95868, 'bjp': 15665, 'who': 103904, 'people': 72687, 'what': 103697, 'congress': 23542, 'like': 56603, 'your': 106068, 'can': 18432, 'only': 69383, 'now': 68191, 'narendra': 65873, 'why': 103994, 'our': 70168, 'govt': 40657, 'its': 49569, 'about': 5191, 'vote': 102364, 'one': 69275, 'dont': 30734, 'him': 43980, 'just': 51979, 'when': 103780, 'their': 95553, 'country': 24824, 'modis': 63217, 'more': 63947, 'how': 45161, 'election': 32535, 'years': 105521, 'rahul': 78736, 'should': 87792, 'time': 96656, 'did': 29225, 'indian': 47231, 'nation': 66048, 'any': 9208, 'against': 6620, 'even': 34101, 'after': 6525, 'then': 95687, 'also': 7771, 'there': 95757, 'which': 103823, 'out': 70190, 'know': 54225, 'governme

In [13]:
from sklearn.model_selection import train_test_split
X = data['Text'].values
Y = data['Score'].values
test_size, val_size = 0.1, 0.05

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size)

print('X train: {}, Y train: {}'.format(X_train.shape, Y_train.shape))
print('X test: {}, Y test: {}'.format(X_test.shape, Y_test.shape))

X train: (146682,), Y train: (146682,)
X test: (16298,), Y test: (16298,)


## Defining different models

In [14]:
# Define a function to create an LSTM model
def lstm_model(num_top_words, input_length):
    # Create a Sequential model
    model = Sequential()
    # Add an Embedding layer with input dimension, output dimension, and input length
    model.add(Embedding(input_dim=num_top_words, output_dim=128, input_length=input_length))
    # Add a CuDNNLSTM layer with 100 units
    model.add(CuDNNLSTM(100))
    # Add a Dense layer with softmax activation for classification
    model.add(Dense(2, activation='softmax'))
    # Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define a function to create a CNN model
def cnn_model(num_top_words, input_length):
    # Create a Sequential model
    model = Sequential()
    # Add an Embedding layer with input dimension, output dimension, and input length
    model.add(Embedding(input_dim=num_top_words, output_dim=128, input_length=input_length))
    # Add a Dropout layer with dropout rate of 0.5
    model.add(Dropout(0.5))
    # Add a 1D Convolutional layer with 128 filters, kernel size of 10, ReLU activation, and L2 regularization
    model.add(Conv1D(128, kernel_size=10, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    # Add a MaxPooling layer
    model.add(MaxPooling1D())
    # Add another Dropout layer with dropout rate of 0.5
    model.add(Dropout(0.5))
    # Flatten the output
    model.add(Flatten())
    # Add a Dense layer with softmax activation for classification and L2 regularization
    model.add(Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
    # Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define a function to create a CNN-LSTM model
def cnn_lstm_model(num_top_words, input_length):
    # Create a Sequential model
    model = Sequential()
    # Add an Embedding layer with input dimension, output dimension, and input length
    model.add(Embedding(input_dim=num_top_words, output_dim=128, input_length=input_length))
    # Add a 1D Convolutional layer with 128 filters, kernel size of 10, and ReLU activation
    model.add(Conv1D(128, kernel_size=10, activation='relu'))
    # Add a MaxPooling layer
    model.add(MaxPooling1D())
    # Add a Dropout layer with dropout rate of 0.5
    model.add(Dropout(0.5))
    # Add a CuDNNLSTM layer with 100 units
    model.add(CuDNNLSTM(100))
    # Add a Dense layer with softmax activation for classification
    model.add(Dense(2, activation='softmax'))
    # Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define a function to create an LSTM-CNN model
def lstm_cnn_model(num_top_words, input_length):
    # Create a Sequential model
    model = Sequential()
    # Add an Embedding layer with input dimension, output dimension, and input length
    model.add(Embedding(input_dim=num_top_words, output_dim=128, input_length=input_length))
    # Add a CuDNNLSTM layer with 100 units and return sequences
    model.add(CuDNNLSTM(100, return_sequences=True))
    # Add a 1D Convolutional layer with 128 filters, kernel size of 10, and ReLU activation
    model.add(Conv1D(128, kernel_size=10, activation='relu'))
    # Add a MaxPooling layer
    model.add(MaxPooling1D())
    # Add a Dropout layer with dropout rate of 0.5
    model.add(Dropout(0.5))
    # Flatten the output
    model.add(Flatten())
    # Add a Dense layer with softmax activation for classification
    model.add(Dense(2, activation='softmax'))
    # Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


## Train models

In [15]:
from datetime import datetime

In [ ]:
# Initialize models dictionary with different model instances
models = { 
    'lstm_model': lstm_model(num_top_words=num_top_words, input_length=X_train.shape[1]), 
    'cnn_model': cnn_model(num_top_words=num_top_words, input_length=X_train.shape[1]),
    'cnn_lstm_model': cnn_lstm_model(num_top_words=num_top_words, input_length=X_train.shape[1]),
    'lstm_cnn_model': lstm_cnn_model(num_top_words=num_top_words, input_length=X_train.shape[1])
}

# Define batch size, number of epochs, and validation size
batch_size = 16
num_epochs = 3
val_size = 0.2

# Iterate over each model in the models dictionary
for name, model in models.items():
    print('Beginning Training', name)
    current_time = datetime.now()

    # Initialize TensorBoard callback for visualization
    tb = TensorBoard(log_dir='./logs/' + name)
    
    # Train the model on training data
    model.fit(
        X_train,  # Training features
        Y_train,  # Training labels
        validation_split=val_size,  # Validation split
        epochs=num_epochs,  # Number of epochs
        batch_size=batch_size,  # Batch size
        callbacks=[tb]  # Callback for TensorBoard
    )
    
    # Print training time for the model
    print("Training", name, "took time", datetime.now() - current_time, "\n\n")


In [ ]:
print(Y_test)

Y_test_new=Y_test[:,1]

print(Y_test_new)

In [ ]:
for name, model in models.items(): 
  # predict probabilities for test set
  yhat_probs = model.predict(X_test, verbose=0)
  # predict crisp classes for test set
  yhat_classes = model.predict_classes(X_test, verbose=0)
  print(name)
  # accuracy: (tp + tn) / (p + n)
  accuracy = accuracy_score(Y_test_new, yhat_classes)
  print('Accuracy: %f' % accuracy)
  # precision tp / (tp + fp)
  precision = precision_score(Y_test_new, yhat_classes)
  print('Precision: %f' % precision)
  # recall: tp / (tp + fn)
  recall = recall_score(Y_test_new, yhat_classes)
  print('Recall: %f' % recall)
  # f1: 2 tp / (2 tp + fp + fn)
  f1 = f1_score(Y_test_new, yhat_classes)
  print('F1 score: %f \n\n' % f1)

lstm_model
Accuracy: 0.871671
Precision: 0.862508
Recall: 0.883328
F1 score: 0.872794 


cnn_model
Accuracy: 0.854227
Precision: 0.847247
Recall: 0.863132
F1 score: 0.855116 


cnn_lstm_model
Accuracy: 0.870509
Precision: 0.868875
Recall: 0.871741
F1 score: 0.870306 


lstm_cnn_model
Accuracy: 0.865135
Precision: 0.868826
Recall: 0.859108
F1 score: 0.863940 


